# Lecture 4 - Basic Statistics I

## Today's Key Takeaways

- Understanding distributions
 - Normal distribution
- Testing for **difference** between two distributions


## Basic Statistics with `scipy` and `statsmodels`

The `scipy` library expands the `numpy` suite of mathematical functions. Like `numpy`, these are broken up into sublibraries. We'll use the `stats` sublibary to run a t-test on our highly variable genes, and use ANOVA on one of those genes. 

Note that we used a new method of importing. Some libraries, like scipy, have multiple sub-libraries. You can import just the sub-library using the `from` command. This saves memory and time. 

We are mostly going to use the stats sublibrary, but we imported all of scipy so we can look over the help file and functions.

For more info about basic statistical testing, go [here](https://machinelearningmastery.com/statistical-hypothesis-tests-in-python-cheat-sheet/) and [here](http://scipy-lectures.org/packages/statistics/index.html).

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import scipy 
from scipy import stats
import statsmodels.stats.multitest as smm
import statsmodels.stats.nonparametric as nonparam 
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import seaborn as sns

In [ ]:
# Setting some pandas preferences
pd.set_option('display.precision', 2)
pd.set_option('display.max_columns',10)

### Start by importing our expression dataset

We're going to start by importing data the expression-metadata merged spreadsheet, so that we've got something to work with.

In [ ]:
# Alternative way to bring in Data
from google.colab import files
uploaded = files.upload()

In [ ]:
# importing the melanoma dataset
melanoma_log2 = pd.read_excel('melanoma_zerosRemoved_log2transformed_2023.xlsx',index_col = 0)
melanoma_log2.head()

We're also going to extract out data frames specific to the different stages, and specific to the cell lines.

In [ ]:
# Extracting out only the gene expression dat from the normal samples and the metastatic samples
normalExp = melanoma_log2.loc[melanoma_log2.Stage == 'primary melanocytes','A1BG':]
metastaticExp = melanoma_log2.loc[melanoma_log2.Stage == 'metastatic','A1BG':]
print(normalExp)
print(metastaticExp)

In [ ]:
# Extracting out only the gene expression data from each of the cell line samples
FMexp = melanoma_log2.loc[melanoma_log2.cell_line == 'FM','A1BG':]
SK28exp = melanoma_log2.loc[melanoma_log2.cell_line == 'SK_MEL_28','A1BG':]
SK147exp = melanoma_log2.loc[melanoma_log2.cell_line == 'SK_MEL_147','A1BG':]
UACCexp = melanoma_log2.loc[melanoma_log2.cell_line == 'UACC_62','A1BG':]

## Assessing the distribution of the data
You can assess the percentiles with the `np.percentile()` function or the `.quantile()` method .

In [ ]:
np.percentile(metastaticExp.A1BG,[10,90])

In [ ]:
metastaticExp.A1BG.quantile([0.1,0.9])

In [ ]:
# Reminder from last time:
# calculates the overall variance df.var() and sorts it in descending order
overall_variance = melanoma_log2.loc[:,'A1BG':].var()
overall_variance.sort_values(inplace = True, ascending= False)
overall_variance.head()

# extract gene names for top 10 most variably expressed genes
topvarGens10 = overall_variance.index[:10]
print(topvarGens10)


## Hypothesis testing: comparing groups of samples

Interpretation:

 - H0: the means of the samples are equal.
 - H1: the means of the samples are unequal.


In [ ]:
# Start by printing out the expression values of the top variable genes
print(normalExp.loc[:,topvarGens10])
print(metastaticExp.loc[:,topvarGens10])

### Comparing two samples, parametric: Student's t-test

Tests whether the means of two independent samples are significantly different.

Assumptions:

- Observations in each sample are independent and identically distributed (iid).
- Observations in each sample are normally distributed.
- Observations in each sample have the same variance.

Let's compare the most variably expressed gene, PMEL.

In [ ]:
# Show what PMEL's expression looks like.
melanoma_log2['PMEL']

In [ ]:
# test the significance of the difference in means with a 2-sample t-test (for independent samples)
stat, p = stats.ttest_ind(normalExp.PMEL, metastaticExp.PMEL)
print(stat) # the t-statistic
print(p) # the p-value

We can test what statistical power we have with making this comparison using the [Power and Sample Size Calculator](http://powerandsamplesize.com/Calculators/Compare-2-Means/2-Sample-Equality). We'll need to calculate as inputs the mean of each group and the standard deviation.

In [ ]:
# using .groupby() to designate groupings by cell line
mel_by_Stage = melanoma_log2.groupby('Stage')

# calculate the means
mel_by_Stage.PMEL.mean()

In [ ]:
# Calculate the standard deviations
mel_by_Stage.PMEL.std()

Note: for comparisons with unequal variances, you can specify `equal_var = False` as an argument in the `ttest_ind()` function.

In [ ]:
# test the significance of the difference in means with a Welch's t-test (to account for unequal standard deviations)
stat, p = stats.ttest_ind(normalExp.PMEL, metastaticExp.PMEL,equal_var = False)
print(p) # the p-value

### <font color=blue> Optional Bonus: Code for Power and Sample Size Calculator implemented in Python

This version of the code calculates a comparion between 2 Means: 2-Sample, 2-Sided Equality [based on implementation here](http://powerandsamplesize.com/Calculators/Compare-2-Means/2-Sample-Equality).

In [ ]:
def SampleSizeCalcGivenPower(meanA, meanB, sd, SampleSizeRatioAB, alpha, power):
  beta=1-power
  nB=(1+1/SampleSizeRatioAB)*(sd*(stats.norm.ppf(1-alpha/2)+stats.norm.ppf(1-beta))/(meanA-meanB))**2
  return math.ceil(nB) # 63

def PowerCalcGivenSampleSize(meanA, meanB, sd, SampleSizeRatioAB, SampleSizeB, alpha):
  z=(meanA-meanB)/(sd*math.sqrt((1+1/SampleSizeRatioAB)/SampleSizeB))
  Power=stats.norm.cdf(z-stats.norm.ppf(1-alpha/2))+stats.norm.cdf(-z-stats.norm.ppf(1-alpha/2))
  return Power

In [ ]:
SampleSizeCalcGivenPower(5, 10, 10, 1, 0.05, 0.8)

In [ ]:
PowerCalcGivenSampleSize(5, 10, 10, 1, 63, 0.05)

### Comparing two samples, non-parametric: Mann-Whitney U-test

Under some circumstances, the assumption for normality may not apply, in which case a non-parametric test might suit better.

To test for the extent to which the data conform to normality, you can use the Shapiro test. Low p-values indicate that the data look different from normality. For more information about testing for normality, go [here](https://www.statology.org/normality-test-python/).

To test for the extent to which the data conform to normality, you can use the Shapiro test. Low p-values indicate that the data look different from normality. For more information about testing for normality, go [here](https://www.statology.org/normality-test-python/).

In [ ]:
# test for normality in the primary cells
stat, p_primary =  stats.shapiro(normalExp.PMEL)
# test for normality in the metastatic cells
stat, p_metastatic =  stats.shapiro(metastaticExp.PMEL)
print(p_primary)
print(p_metastatic)

Given that one of the groups noes not look super normal, worth considering using the non-parametric comparison test instead of the t-test.

The Mann-Whitney U-test assesses whether the distributions of two independent samples are equal or not.

Assumptions:

- Observations in each sample are independent and identically distributed (iid).
- Observations in each sample can be ranked.


In [ ]:
# non-parametric test for significance of the difference in rank 
stat, p = stats.mannwhitneyu(normalExp.PMEL, metastaticExp.PMEL)
print(p)


### <font color=brown>Hands on practice</font>

Find the gene with the highest overall mean expression. Use the Student's t-test to assess whether the gene's expression is different between the metastatic and the normal melanocyte samples. 

1. What is the resulting p-value? Is the expression significantly different between the two groups?

2. What was the statistical power for this comparison? 

### <font color=blue> Optional Bonus: Comparing multiple samples, parametric -  Analysis of Variance Test (ANOVA)

Tests whether the means of two or more independent samples are significantly different.

Assumptions:

- Observations in each sample are independent and identically distributed (iid).
- Observations in each sample are normally distributed.
- Observations in each sample have the same variance.

In [ ]:
# ANOVA lets us see if any one of the cell lines has a significant difference in the mean
stat, p = stats.f_oneway(FMexp.PMEL, SK147exp.PMEL, SK28exp.PMEL, UACCexp.PMEL)
p

### <font color=blue> Optional Bonus: Comparing multiple samples, non parametric - Kruskal-Wallis H Test

Tests whether the distributions of two or more independent samples are equal or not.

Assumptions:

- Observations in each sample are independent and identically distributed (iid).
- Observations in each sample can be ranked.


In [ ]:
# The K-W H test lets us see if any one of the cell lines has a significant difference in the mean
stat, p = stats.kruskal(FMexp.PMEL, SK147exp.PMEL, SK28exp.PMEL, UACCexp.PMEL)
p